# 사전 학습된 BERT 모델로 질문에 대한 답변을 제시하는 예제

## 사전에 필요한 라이브러리 다운로드

In [ ]:
# 0. 사전에 필요한 라이브러리 다운로드
!pip install transformers

## AutoModelForQuestionAnswering:

- 사전 학습된 질문-답변 모델을 불러오기 위한 라이브러리
- 예: SQuAD(SQuAD 2.0 등) 데이터셋으로 학습된 BERT 모델 사용

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

## bert-large-uncased-whole-word-masking-finetuned-squad 모델과 Tokenizer

- 사용할 사전 학습된 모델의 이름을 지정
  - **`bert-large`**: BERT의 대형 버전으로, 더 큰 크기의 모델(3억 4천만 개의 파라미터)을 사용.
  - **`uncased`**: 영어 텍스트에서 대소문자를 구분하지 않음.
  - **`whole-word-masking`**: BERT 모델이 사전 학습 중 단어 전체를 마스킹하는 방식.
  - **`finetuned-squad`**: SQuAD 데이터셋에 대해 파인튜닝된 질문-답변 모델.

- Tokenizer 역할: 입력 데이터 전처리
  - 텍스트를 토큰으로 분리(예: `"Transformers"` → `['transform', '##ers']`).
  - 토큰을 정수 ID로 매핑(예: `['transform', '##ers']` → `[19204, 2900]`).

In [ ]:
# 1. 모델과 토크나이저 로드
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

##  질문과 컨텍스트 설정

In [ ]:
# 2. 질문과 컨텍스트 설정
context = """
BERT (Bidirectional Encoder Representations from Transformers) is a transformer-based machine learning model
for natural language processing (NLP) pre-training developed by Dongle(Dongjun & Google). BERT was created and published in 2018
by Jacob Devlin and his colleagues from Dongle.
"""

question = "Who developed BERT?"

## 입력 토큰화

문장을 토큰화하고, 특수 토큰([CLS], [SEP])을 추가한 결과는 다음과 같다.

| **토큰**    | **[CLS]** | **who**  | **developed** | **bert**  | **?**    | **[SEP]** |
| ----------- | --------- | -------- | ------------- | --------- | -------- | --------- |
| **정수 ID** | **101**   | **2040** | **2764**      | **14324** | **1029** | **102**   |


In [ ]:
# 3. 입력 토큰화
inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)

# inputs
print(inputs)

## 모델로 예측

In [ ]:
# 4. 모델로 예측
with torch.no_grad():
    outputs = model(**inputs)

## 예측된 시작과 끝 위치를 가져오기

- 확률 관점에서 가장 가능성이 높은 답변을 추출 가능
- 확률 기반 접근은 자연어 처리의 모호성과 문맥 의존성을 효과적으로 처리 가능

In [ ]:
# 5. 예측된 시작과 끝 위치를 가져오기
start_scores = outputs.start_logits
end_scores = outputs.end_logits

start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

## 토큰을 문자열로 디코딩하여 답변 생성
- Dongle로 정확한 답변을 제시

In [ ]:
# 6. 토큰을 문자열로 디코딩하여 답변 생성
answer = tokenizer.decode(inputs["input_ids"][0][start_index : end_index + 1])

print(f"Question: {question}")
print(f"Answer: {answer}")